In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" 
#Change the path of some HF environment variables to store the download data (model and dataset) from the hub to a choosen location
os.environ['HF_HOME'] = "/home/rbencharef/Experiments/DocVQA/DIVE_Doc/.cache"
os.environ['HF_HUB_CACHE'] = "/home/rbencharef/Experiments/DocVQA/DIVE_Doc/.cache"
os.environ['HF_DATASETS_CACHE'] = "/home/rbencharef/Experiments/DocVQA/DIVE_Doc/.cache"
from transformers import PaliGemmaProcessor, AutoTokenizer, DonutImageProcessor
from modeling_divedoc import DIVEdoc
from processing_divedoc import PaliGemmaProcessor
from datasets import load_dataset

/home/rbencharef/miniconda3/envs/new_hf_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = DIVEdoc.from_pretrained(
    "JayRay5/DIVE-Doc-FRD",
    trust_remote_code=True)

Vision config in end-to-end model: swinpam
[64, 64]
[64, 64]


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 17.10it/s]


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-ft-docvqa-896")
image_processor = DonutImageProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
image_processor.image_seq_length = 4096

In [4]:
processor = PaliGemmaProcessor(tokenizer=tokenizer, image_processor=image_processor)

In [5]:
test_dataset = load_dataset("pixparse/docvqa-single-page-questions", split="test",streaming=True)
batch_dataset = test_dataset.batch(2) 
iterative_dataset = iter(batch_dataset)
batch = next(iterative_dataset)

In [18]:
#preprocessing 
imgs = []
for i in batch["image"]:
    imgs.append(i.convert('RGB'))
txt = batch["question"]
model_inputs = processor(text=txt, images=imgs, return_tensors="pt",padding=True)
input_len = model_inputs["input_ids"].shape[-1]
    

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


In [17]:
batch["question"]

['Routing#?', 'Heading of the page?']

In [12]:
model_inputs["input_ids"].shape

torch.Size([1, 4109])

In [7]:
imgs = []
for i in batch["image"]:
    imgs.append(i.convert('RGB'))
processor.image_processor(images=imgs[0])

{'pixel_values': [array([[[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]]],
      shape=(3, 2560, 1920), dtype=float32)]}